# Conexão com drive

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Atividade 1: Remoção de comentários do BTGPactual

Este processo é essencial pois não queremos uma seleção enviada no pré processamento, e entendemos que os comentários do Instagram do BTG Pactual não irá comentar de forma negativa nenhuma de duas postagens.

### 1.1.1. Setup

In [24]:
pip install unidecode


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [25]:
pip install -U emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [26]:
!pip install emoji --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [27]:
import pandas as pd
from textblob import TextBlob
import emoji
import re
import unidecode

Pandas: é uma biblioteca de análise de dados em Python que oferece estruturas de dados e ferramentas para manipulação e análise de dados. Com o Pandas, é possível ler, escrever e manipular dados em diversos formatos, como CSV, Excel, SQL, etc. Ele oferece uma grande quantidade de funcionalidades para trabalhar com dados em Python, incluindo a capacidade de filtrar, agregar, reorganizar e transformar dados de várias maneiras.

TextBlob: é uma biblioteca de processamento de linguagem natural em Python. Ela oferece uma interface simples para tarefas comuns de NLP, como análise de sentimentos, correção ortográfica, extração de frases-chave e classificação de texto. O TextBlob é construído sobre a biblioteca NLTK (Natural Language Toolkit) e oferece uma sintaxe fácil de usar para muitas tarefas de processamento de linguagem natural. É uma biblioteca muito útil para análise de texto em Python.

Emoji: é uma biblioteca Python que permite a conversão de emojis de texto para representações de texto Unicode e vice-versa. Além disso, fornece funções úteis para trabalhar com emojis, como contar o número de emojis em uma string, remover todos os emojis de uma string e substituir cada emoji em uma string por um texto de substituição especificado.

Re: A biblioteca "re" é uma ferramenta para trabalhar com expressões regulares, permitindo a extração e manipulação de informações de forma eficiente. 

Unidecode: Unicode é um padrão de codificação de caracteres de forma universal para a representação de caracteres de todas as línguas escritas, bem como símbolos matemáticos, musicais, entre outros.

### 1.1.2. Lendo CSV

In [28]:
dados = pd.read_csv('/content/drive/MyDrive/baseDadosBTG/BaseBTG.csv')

In [29]:
#retirando aspas dos nomes das colunas
dados = dados.rename(columns=lambda x: re.sub('[\"\']', '', x))
dados

,id,dataPublicada,autor,texto,sentimento,tipoInteracao,anomalia,probabilidadeAnomalia,linkPost,processado,contemHyperlink
0,1,"""2022-03-04 09:38:00""",v8_capital,Confira os resultados dos nossos fundos no mês...,NEUTRAL,marcação,1,52,https://www.instagram.com/p/CarqO6drRux/,0,0
1,2,"""2022-03-04 09:29:00""",winthegame_of,A Alvarez & Marsal estará conosco no Sportainm...,NEUTRAL,marcação,0,43,https://www.instagram.com/p/CarpQUluh7z/,0,0
2,3,"""2022-03-04 09:20:00""",marta_bego,#Repost btgpactual with make_repost ・・・ Entend...,NEUTRAL,marcação,0,45,https://www.instagram.com/p/CaroK9Nu8tC/,0,0
3,4,"""2022-03-04 08:36:00""",lmviapiana,Minuto touro de ouro,POSITIVE,comentário,0,38,https://www.instagram.com/p/CapXhB5Lvas/#17977...,0,0
4,5,"""2022-03-03 21:10:00""",vanilson_dos,@ricktolledo Sim,NEUTRAL,resposta,0,17,https://www.instagram.com/p/CapXhB5Lvas/#17842...,0,0
...,...,...,...,...,...,...,...,...,...,...,...
12350,3044013,"""2022-11-30 11:51:00""",onixcapital,Uma noite de encontros e muito conhecimento on...,NEUTRAL,marcação,1,54,https://www.instagram.com/reel/CllsqI7goSC/,0,0
12351,3044212,"""2022-11-30 12:25:00""",amgcapitalinvest,Erros financeiros para eliminar antes dos 30 a...,NEUTRAL,marcação,0,39,https://www.instagram.com/p/Cllw0CmNGkX/,0,0
12352,3044213,"""2022-11-30 12:18:00""",jayipacderota,Estou muito grato por todo o esforço que você ...,POSITIVE,comentário,1,52,https://www.instagram.com/p/CllkSBOuKlJ/#17960...,0,0
12353,3044412,"""2022-11-30 12:50:00""",moinho_cultural,"""👉GUADAKAN está chegando!!!! A aula pública e...",NEUTRAL,marcação,0,43,https://www.instagram.com/reel/ClluX-cAoaf/,0,0


### 1.1.2. Teste Isolado

### 1.1.3. Definição de Função

Função de remoção de autor

In [30]:
def filtrar_dados_por_autor(dados, autor):
    # Filtra os dados por autor, removendo as linhas em que a coluna 'autor' contém a string passada como parâmetro.
    # Substitui os valores nulos (NaN) da coluna 'autor' por 'desconhecido'.
    # Retorna um novo DataFrame com as linhas filtradas.
    # Substitui os valores nulos (NaN) da coluna 'autor' por 'desconhecido'.
    dados['autor'] = dados['autor'].fillna('desconhecido')

    # Cria um novo DataFrame chamado 'dados_filtrados', que contém todas as linhas do DataFrame original (dados), exceto aquelas em que a coluna 'autor' contém a string passada como parâmetro.
    # O operador ~ é usado para selecionar todas as linhas em que a condição 'dados['autor'].str.contains(autor)' é falsa.
    dados_filtrados = dados[~dados['autor'].str.contains(autor)]
    
    return dados_filtrados

#### 1.1.3.1. Testes da Função de Remoção de autor

In [31]:
dados_filtrados = filtrar_dados_por_autor(dados, "autor")
print(dados_filtrados)

            id          dataPublicada             autor  \
0            1  "2022-03-04 09:38:00"        v8_capital   
1            2  "2022-03-04 09:29:00"     winthegame_of   
2            3  "2022-03-04 09:20:00"        marta_bego   
3            4  "2022-03-04 08:36:00"        lmviapiana   
4            5  "2022-03-03 21:10:00"      vanilson_dos   
...        ...                    ...               ...   
12350  3044013  "2022-11-30 11:51:00"       onixcapital   
12351  3044212  "2022-11-30 12:25:00"  amgcapitalinvest   
12352  3044213  "2022-11-30 12:18:00"     jayipacderota   
12353  3044412  "2022-11-30 12:50:00"   moinho_cultural   
12354  3045012  "2022-11-30 14:27:00"         bfmarilia   

                                                   texto sentimento  \
0      Confira os resultados dos nossos fundos no mês...    NEUTRAL   
1      A Alvarez & Marsal estará conosco no Sportainm...    NEUTRAL   
2      #Repost btgpactual with make_repost ・・・ Entend...    NEUTRAL   
3      

### 1.1.4. Tratamento de dados nulos

In [32]:
# identifica células com valores nulos
dados.isnull()

# conta o número de valores nulos em cada coluna
dados.isnull().sum()

id                       0
dataPublicada            0
autor                    0
texto                    0
sentimento               0
tipoInteracao            0
anomalia                 0
probabilidadeAnomalia    0
linkPost                 0
processado               0
contemHyperlink          0
dtype: int64

Após essa operação necessitamos ver na coluna texto - onde estão os comentários - as linhas vazias para que seja substituída por uma string vazia para que possamos analisar o sentimento das palavras.

In [33]:
# conta o número total de valores nulos na coluna texto do DataFrame
dados['texto'].isnull().sum().sum()

0

Nesse código, está sendo preenchido com uma string vazia os valores nulos (NaN) da coluna 'texto' do DataFrame 'dados_filtrados'. Isso é feito com o método 'fillna' do pandas, que substitui os valores nulos pelo valor especificado como argumento, no caso, uma string vazia ('').

O resultado da operação é atribuído à variável 'dados_depois', mas também é atribuído ao próprio DataFrame 'dados_filtrados' (de forma redundante), com a finalidade de atualizar o valor no DataFrame original.

In [34]:
# preenche os valores vazios na coluna 'texto' com uma string vazia
dados_filtrados['texto'] = dados_filtrados['texto'].fillna('')

# Atividade 3: Descrição de emojis de Emojis

### 3.1.2. Descrição da função

In [59]:
# define a função de remoção de emojis
def removeEmoji():
    remove_emoji = lambda x: emoji.demojize(str(x)).replace(":", "").strip()
    return remove_emoji

### 3.1.3. Teste da função

In [60]:
# aplica a função à coluna "texto" do DataFrame

dados['textoLimpo'] = dados['texto'].apply(removeEmoji())
dados['textoLimpo']

0        Confira os resultados dos nossos fundos no mês...
1        A Alvarez & Marsal estará conosco no Sportainm...
2        #Repost btgpactual with make_repost ・・・ Entend...
3                                     Minuto touro de ouro
4                                         @ricktolledo Sim
                               ...                        
12350    Uma noite de encontros e muito conhecimento on...
12351    Erros financeiros para eliminar antes dos 30 a...
12352    Estou muito grato por todo o esforço que você ...
12353    "backhand_index_pointing_rightGUADAKAN está ch...
12354    @btgpactual pq o morning call não está mais ap...
Name: textoLimpo, Length: 12355, dtype: object